In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
path ='data1'

In [3]:
fund_returnA =  pd.read_csv(path+'/test_fund_return.csv')
fund_returnB =  pd.read_csv(path+'/train_fund_return.csv')
fund_return = pd.merge(fund_returnB,fund_returnA,how='left')

In [4]:
benchmark_returnA =  pd.read_csv(path+'/test_fund_benchmark_return.csv')
benchmark_returnB =  pd.read_csv(path+'/train_fund_benchmark_return.csv')
benchmark_return = pd.merge(benchmark_returnB,benchmark_returnA,how='left')

In [5]:
test_correlation = pd.read_csv(path+'/test_correlation.csv')
train_correlation = pd.read_csv(path+'/train_correlation.csv')
test = pd.merge(train_correlation,test_correlation,how='left')

In [6]:
ID = test['Unnamed: 0']
test.drop(['Unnamed: 0'],axis=1,inplace=True)

In [7]:
index = pd.concat([ID.map(lambda x:x.split('-')[0]),ID.map(lambda x:x.split('-')[1])],axis=1)
index.columns = ['FundA','FundB']

In [16]:
def lgbPRD(x_train,target,val,val2,x_test,params,num_boost_round=10000):
    import lightgbm as lgb
    #xgb矩阵转换
    xg_train = lgb.Dataset(np.array(x_train),target)


    bst = lgb.train(params, xg_train,num_boost_round)
    
    
    return bst.predict( x_test ),bst.predict( val ),bst.predict( val2 )

In [8]:
def day_tran(data,star_time,end_time,index=index):
    fund_r = pd.concat((data['Unnamed: 0'],data[data.columns[star_time:end_time]]),axis=1)
    fund_r.rename(columns={'Unnamed: 0':"FundA"},inplace=True)
    FundA  = pd.merge(index,fund_r,how = 'left')      
    FundA = FundA[FundA.columns[2:]]
    FundA.columns=range(0,FundA.shape[1])
    
    fund_r.rename(columns={"FundA":"FundB"},inplace=True)
    FundB = pd.merge(index,fund_r,on = 'FundB',how = 'left',suffixes=('_x', '_y'))
    FundB = FundB[FundB.columns[2:]]
    FundB.columns=range(0,FundB.shape[1])
    return FundA,FundB

In [17]:
#定义结果评价函数，可通过倒数第2天的correlation数据，预测倒数第1天的correlation，以验证模型效果
from sklearn.metrics import mean_absolute_error  
def modelFeval(y_pred,labels):
    sc1 = mean_absolute_error(labels,y_pred)
    diff = 0
    count = 0
    for i in range(len(y_pred)):
        diff += abs((y_pred[i]-labels[i]) / (1.5-labels[i]))
        count +=1
    sc2 = diff/count
    sc = ((2/(2+sc1+sc2))**2)
    return 'myFeval',float(sc)

In [9]:
day = 300

In [10]:
for i in tqdm(range(5)):    
    FundA,FundB = day_tran(fund_return,-day-63,-63)
    testA = test.mean(axis=1)
    FundAA = FundA.mean(axis=1)
    FundBB =FundB.mean(axis=1)
    x_train = pd.concat([abs(FundA-FundB),testA,FundAA,FundBB,abs(FundAA-FundBB)],axis=1)
    
    if i ==0 :
        xtrain = x_train
        ytrain = test[test.columns[-3]]
    else:
        xtrain = pd.concat([xtrain,x_train])
        ytrain = pd.concat([ytrain,test[test.columns[-3-i]]])

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:52<00:00, 10.51s/it]


In [11]:
FundA,FundB = day_tran(fund_return,-day,None)
testA = test.mean(axis=1)
FundAA = FundA.mean(axis=1)
FundBB =FundB.mean(axis=1)
x_test = pd.concat([abs(FundA-FundB),testA,FundAA,FundBB,abs(FundAA-FundBB)],axis=1)
q = x_test.mean()
x_test = x_test.fillna(0).replace(0,q)

FundA,FundB = day_tran(fund_return,-day-61,-61)
testA = test.mean(axis=1)
FundAA = FundA.mean(axis=1)
FundBB =FundB.mean(axis=1)
xval = pd.concat([abs(FundA-FundB),testA,FundAA,FundBB,abs(FundAA-FundBB)],axis=1)
q = xval.mean()
xval1 = xval.fillna(0).replace(0,q)

FundA,FundB = day_tran(fund_return,-day-62,-62)
testA = test.mean(axis=1)
FundAA = FundA.mean(axis=1)
FundBB =FundB.mean(axis=1)
xval = pd.concat([abs(FundA-FundB),testA,FundAA,FundBB,abs(FundAA-FundBB)],axis=1)
q = xval.mean()
xval2 = xval.fillna(0).replace(0,q)

In [12]:
xtrain.shape,ytrain.shape,x_test.shape,xval1.shape,xval2.shape

((1305015, 304), (1305015,), (261003, 304), (261003, 304), (261003, 304))

In [13]:
lgb_params = {
    'device':'gpu',
    'application':'regression_l1',
    'seed': 0,
    'learning_rate':0.04,
    'max_depth':1,
    'feature_fraction':0.8,
    'lambda_l1':1,
}

In [18]:
lgypred,lgyval1,lgyval2 = lgbPRD(xtrain,ytrain,xval1,xval2,x_test,lgb_params,900)

In [ ]:
modelFeval(lgyval1,test[test.columns[-1]])

In [19]:
df = pd.DataFrame({'ID':ID,'value':lgypred})

In [20]:
df.to_csv('base.csv',index=None)

In [ ]:
df